## ひろしまQuest2020#stayhome【球種予測部門】
#### all_merge_21
- 19相当に戻した

#### all_merge_20
- player同士の組み合わせを削除

#### all_merge_19
- コースの種類のみ

#### all_merge_18
- コースの種類、1球前のボール、ファウル数

#### all_merge_17
- イニング時間、打席時間、平均投球間隔(差)、サヨナラ、延長戦

#### all_merge_16
- 球団ごとの年棒ランク
- 選手IDごとの2017年のコース実績

#### all_merge_15
- 左右で球種/コースを別に集計

#### all_merge_14
- 左右で球種を別に集計

#### all_merge_13
- 前の投球からの時間差

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 300)

### 管理番号

In [2]:
model_No = 21

### sub-modelを使用するかどうか

In [3]:
use_sub_model = True

In [4]:
ALL_MERGE = 'intermediate/all_merge_{}.f'.format(model_No)
SUBMIT = 'submit/ball_{}.csv'.format(model_No)

if use_sub_model:
    ALL_MERGE = 'intermediate/all_merge_{}_sub.f'.format(model_No)
    SUBMIT = 'submit/ball_{}_select5.csv'.format(model_No)

OUT_SUBMODEL = 'intermediate/ball_predict_{}.f'.format(model_No)

In [5]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 211)

In [6]:
if use_sub_model:
    all_pitch['predict_high_str'] = all_pitch['predict_0'] + all_pitch['predict_3'] + all_pitch['predict_6'] 
    all_pitch['predict_high_ball'] = all_pitch['predict_9'] + all_pitch['predict_10'] 
    all_pitch['predict_mid_str'] = all_pitch['predict_1'] + all_pitch['predict_4'] + all_pitch['predict_7'] 
    all_pitch['predict_low_str'] = all_pitch['predict_2'] + all_pitch['predict_5'] + all_pitch['predict_8'] 
    all_pitch['predict_low_ball'] = all_pitch['predict_11'] + all_pitch['predict_12'] 

    all_pitch['predict_left_str'] = all_pitch['predict_0'] + all_pitch['predict_1'] + all_pitch['predict_2'] 
    all_pitch['predict_left_ball'] = all_pitch['predict_9'] + all_pitch['predict_11'] 
    all_pitch['predict_center_str'] = all_pitch['predict_3'] + all_pitch['predict_4'] + all_pitch['predict_5'] 
    all_pitch['predict_right_str'] = all_pitch['predict_6'] + all_pitch['predict_7'] + all_pitch['predict_8'] 
    all_pitch['predict_right_ball'] = all_pitch['predict_10'] + all_pitch['predict_12'] 

In [7]:
if use_sub_model:
    all_pitch.drop(columns=[
        'predict_straight', 'predict_curve', 'predict_slider', 'predict_shoot',
        'predict_fork', 'predict_changeup', 'predict_sinker', 'predict_cutball',
        'predict_0','predict_1','predict_2','predict_3','predict_4','predict_5','predict_6',
        'predict_7','predict_8','predict_9','predict_10','predict_11','predict_12'
    ], inplace=True)

In [8]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 200)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,elapsed_batter,ave_elapsed_game,ave_elapsed_inning,ave_elapsed_batter,diff_elapsed_batter,diff_elapsed_inning,diff_elapsed_game,sayonara,extention,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_count_sum,pre_ball_foul,pre_ball_ball,pre_ball_strike,pre_foul_sum,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_rank_pit,rank_year_pit,rank_x_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,high_str,high_ball,mid_str,low_str,low_ball,left_str,left_ball,center_str,right_str,right_ball,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_rank_bat,rank_year_bat,rank_x_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R,predict_high_str,predict_high_ball,predict_mid_str,predict_low_str,predict_low_ball,predict_left_str,predict_left_ball,predict_center_str,predict_right_str,predict_right_ball
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.237468,0.325101,0.209547,0.145227,0.092978,0.027829,0.113620,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,0.190270,0,0,0,0,0,1,0,0,0.000000,NaN,0,0.000000,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.468793,0.421637,0.230236,0.041609,0.109570,0.0,0.0,1638.0,27.0,190.0,404.0,0.156899,0.202686,0.214896,0.180098,0.245421,0.186813,0.230769,0.188645,0.176435,0.217338,0,176,74,5.0,15000,0,1,1,0,32,8.0,2.5,0.312500,20.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0,0.197440,0.193418,0.263644,0.183228,0.162270,0.263206,0.206331,0.228361,0.152745,0.149357
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.210923,0.247198,0.177502,0.320359,0.147952,0.039943,0.116772,0.106283,0.230235,0.157397,0.138195,0.367890,0.184621,0.355720,0.111331,0.105923,0.242405,0,0,0,0,0,1,0,0,0.200000,0.200000,0,0.200000,2,0.200000,0.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0.0,0.0,1.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.468793,0.421637,0.230236,0.041609,0.109570,0.0,0.0,1638.0,

### train
- 行数: 257117

In [9]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 200)

### test
- 行数: 521650

In [10]:
test = all_pitch[all_pitch['course'].isnull()]
test.shape

(521650, 200)

In [11]:
del all_pitch
gc.collect()

60

In [12]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

In [13]:
test_d = test.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [14]:
lgb_param = {
    'objective' : 'multiclass',
    'boosting_type': 'gbdt',
    'metric' : 'multi_logloss',
    'num_class' : 8,
    'seed' : 0,
    'learning_rate' : 0.1,
    'lambda_l1': 6.9923570049658075, 
    'lambda_l2': 0.002378623984798833, 
    'num_leaves': 18, 
    'feature_fraction': 0.45199999999999996, 
    'bagging_fraction': 0.9799724836460725, 
    'bagging_freq': 4, 
    'min_child_samples': 20
}

In [15]:
def lightgbm_cv(lgb_param, lgb_train):
    cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

    num_boost_round = len(cv_results['multi_logloss-mean'])
    print('Best num_boost_round:', num_boost_round)
    best_cv_score = cv_results['multi_logloss-mean'][-1]
    print('Best CV score:', best_cv_score)
    best_iter = int(num_boost_round * 1.1)
    return best_iter

### Feature Importance

In [16]:
def feature_importance(lgb_model):
    fi = lgb_model.feature_importance()
    fn = lgb_model.feature_name()
    df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
    df_feature_importance.sort_values('feat_imp', inplace=True)
    return df_feature_importance

In [17]:
%%time
lgb_train = lgb.Dataset(train_d, train['ball'])
best_iter = lightgbm_cv(lgb_param, lgb_train)

[100]	cv_agg's multi_logloss: 1.13859 + 0.00237025
[200]	cv_agg's multi_logloss: 1.11632 + 0.0025556
[300]	cv_agg's multi_logloss: 1.10977 + 0.00245328
[400]	cv_agg's multi_logloss: 1.10598 + 0.00242949
[500]	cv_agg's multi_logloss: 1.10342 + 0.00246199
[600]	cv_agg's multi_logloss: 1.10174 + 0.0025586
[700]	cv_agg's multi_logloss: 1.10037 + 0.00257971
[800]	cv_agg's multi_logloss: 1.09954 + 0.00259129
[900]	cv_agg's multi_logloss: 1.09899 + 0.0027595
[1000]	cv_agg's multi_logloss: 1.0987 + 0.00286267
[1100]	cv_agg's multi_logloss: 1.09863 + 0.00299436
[1200]	cv_agg's multi_logloss: 1.09857 + 0.00309194
Best num_boost_round: 1183
Best CV score: 1.098530202050569
CPU times: user 2h 11min 57s, sys: 38.5 s, total: 2h 12min 36s
Wall time: 8min 18s


### Feature Importanceの下位10個を落とす×5

In [18]:
%%time
for i in range(5):
    lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)
    
    df_fi = feature_importance(lgb_model)
    print(df_fi.head(10))
    fi_last = df_fi[0:10]['feat_name']
    
    train_d = train_d.drop(fi_last, axis=1)
    test_d = test_d.drop(fi_last, axis=1)
    lgb_train = lgb.Dataset(train_d, train['ball'])

        feat_name  feat_imp
40        pos_pit         0
147     foreigner         0
142          firm         0
81   ball_cnt_2-0         0
123      firm_bat         0
85       firm_pit         0
16         bat_LR         1
80   ball_cnt_1-3         1
73   ball_cnt_0-0         1
186   pit_bat_R_R         3
        feat_name  feat_imp
73   ball_cnt_0-3         3
72   ball_cnt_0-2         7
53      extention         7
174   pit_bat_L_L        10
38           role        11
79   ball_cnt_2-3        19
71   ball_cnt_0-1        19
52       sayonara        22
175   pit_bat_L_R        25
76   ball_cnt_1-2        26
         feat_name  feat_imp
166    pit_bat_R_L        21
113  foreigner_bat        26
61        fourball        29
79     highsch_pit        34
70    ball_cnt_2-1        37
15        pitch_LR        38
116    highsch_bat        39
78        univ_pit        40
68    ball_cnt_1-0        40
76   foreigner_pit        49
        feat_name  feat_imp
67   ball_cnt_2-2        33
58       

### 再学習

In [19]:
%%time
best_iter = lightgbm_cv(lgb_param, lgb_train)
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

[100]	cv_agg's multi_logloss: 1.1375 + 0.00249962
[200]	cv_agg's multi_logloss: 1.11645 + 0.00251288
[300]	cv_agg's multi_logloss: 1.10993 + 0.00264126
[400]	cv_agg's multi_logloss: 1.10639 + 0.00262861
[500]	cv_agg's multi_logloss: 1.104 + 0.00269934
[600]	cv_agg's multi_logloss: 1.1023 + 0.00281394
[700]	cv_agg's multi_logloss: 1.10104 + 0.00282489
[800]	cv_agg's multi_logloss: 1.10015 + 0.00290852
[900]	cv_agg's multi_logloss: 1.09962 + 0.00292341
[1000]	cv_agg's multi_logloss: 1.09918 + 0.00304609
[1100]	cv_agg's multi_logloss: 1.09903 + 0.0030904
[1200]	cv_agg's multi_logloss: 1.09897 + 0.00320578
Best num_boost_round: 1172
Best CV score: 1.0989019423650523
CPU times: user 2h 31min 3s, sys: 42.4 s, total: 2h 31min 45s
Wall time: 9min 30s


In [20]:
feature_importance(lgb_model).tail(30)

,feat_name,feat_imp
66,curve,1614
132,salary_year_dif_b-c,1647
135,rank_x_year_dif_b-c,1790
99,batter_cnt,1879
67,slider,1935
120,bmi_dif_p-b,2023
0,pitch_cnt_in_game,2080
146,predict_right_ball,2090
35,elapsed_min,2127
136,bmi_dif_b-c,2212


### 予測

In [21]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 7min 27s, sys: 576 ms, total: 7min 28s
Wall time: 29.2 s


## 結果出力

In [22]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.455985,0.045721,0.102740,0.318445,0.002014,0.000009,0.000014,0.075073
1,1,0.406549,0.086383,0.088004,0.371341,0.002912,0.000011,0.000005,0.044794
2,2,0.375990,0.102923,0.117031,0.146178,0.132901,0.000078,0.000030,0.124869
3,3,0.414788,0.079917,0.124767,0.302783,0.022509,0.000023,0.000017,0.055196
4,4,0.162038,0.062528,0.280713,0.406052,0.014514,0.000021,0.000012,0.074121


In [23]:
submit.to_csv(SUBMIT, header=False, index=False)

### コース予測で使用

In [24]:
%%time
if not use_sub_model:
    train_predict = lgb_model.predict(train_d, num_iteration = lgb_model.best_iteration)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


In [25]:
if not use_sub_model:
    df_train_predict = pd.DataFrame(train_predict).reset_index()
    submodel = pd.concat([df_train_predict, submit], ignore_index=True)
    submodel.drop(columns=['index'], inplace=True)
    submodel.rename(columns={
        0: 'predict_straight',
        1: 'predict_curve',
        2: 'predict_slider',
        3: 'predict_shoot',
        4: 'predict_fork',
        5: 'predict_changeup',
        6: 'predict_sinker',
        7: 'predict_cutball'
    }, inplace=True)
    print(submodel.shape)
    submodel.head()

In [26]:
if not use_sub_model:
    submodel.to_feather(OUT_SUBMODEL)

In [27]:
column_cnt = len(train_d.columns)

### SIGNATE

In [29]:
'signate submit --competition-id=275 ./{} --note feat={}_cv='.format(SUBMIT, column_cnt)

'signate submit --competition-id=275 ./submit/ball_21_select5.csv --note feat=147_cv='

### GCS

In [ ]:
'gsutil cp ./{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

### 評価結果
#### 21_select (147) ... FI下位50個を削除
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1172
- Best CV score: 1.0989019423650523
- Wall time: 9min 30s
- 評価結果  : 1.4009009

#### 21_sub (208) ... 球種のサブモデルの結果をそのまま使った
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1021
- Best CV score: 0.812901550823909
- Wall time: 8min 17s
- 評価結果  : 1.6952410

#### 20_sub (176) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1328
- Best CV score: 1.097496116910761
- Wall time: 14min 8s -> 8min 26s(n1-standard-16)
- 評価結果  : 1.4025144

#### 20 (166) ... player同士の組み合わせを削除
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1263
- Best CV score: 1.098749368845937
- Wall time: 12min 30s
- 評価結果  : 1.3964075

#### 19_sub (200) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1199
- Best CV score: 1.098298504761071
- Wall time: 14min 16s -> 7min 23s(n1-standard-16)
- 評価結果  : 1.3945102 ★

#### 19 (190) ... コースの種類のみ
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1141
- Best CV score: 1.1004105063817775
- Wall time: 12min 53s
- 評価結果  : 1.3974324

#### 18_sub (230) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.1004215512272495
- Wall time: 15min 34s
- 評価結果  : 1.3953962

#### 18 (217) ... コースの種類、1球前のボール、ファウル数
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1102
- Best CV score: 1.1004234715781385
- Wall time: 13min 42s
- 評価結果  : 1.3954167

#### 17 (191) ... イニング時間、打席時間、平均投球間隔(差)、サヨナラ、延長戦
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1034107856339535
- Wall time: 26min 44s
- 評価結果  : 1.3963057

#### 16 (181) ... 選手IDごとの2017年のコース実績
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1178
- Best CV score: 1.1021930539837317
- Wall time: 12min 35s
- 評価結果  : 1.3993836

#### 15 (153) ... 左右で球種/コースを別に集計
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1189
- Best CV score: 1.1013869435441306
- Wall time: 11min 9s
- 評価結果  : 未提出

#### 14 (153) ... 左右で球種を別に集計
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1240
- Best CV score: 1.1021952294615331
- Wall time: 11min 34s
- 評価結果  : 1.3981900

#### 13 (157) ... 前の投球からの時間差を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1416
- Best CV score: 1.1062463941695753
- Wall time: 13min 10s
- 評価結果  : 1.4012184

#### 12 (156) ... コースの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1353
- Best CV score: 1.106942086205103
- Wall time: 27min 41s
- 評価結果  : 1.4016077

#### 11 (143) ... ボールカウントの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1209
- Best CV score: 1.1075153721728936
- Wall time: 23min 19s
- 評価結果  : 未提出

#### 10_tune (135) ... 再チューニング
- 'num_leaves' : 15,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1260
- Best CV score: 1.1118897437428692
- Wall time: 8min 40s
- 評価結果  : 1.4060397

#### 10s (135) ... trainを75%サンプリング
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1412
- Best CV score: 1.1190287120710383
- Wall time: 12min 5s
- 評価結果  : 1.4310869

#### 10 (135) ... 5相当に戻した
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1112
- Best CV score: 1.111265000190272
- Wall time: 9min 30s
- 評価結果  : 1.4008712

#### 9 (136) ... 5相当に戻した=>NG
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1106
- Best CV score: 1.1113177029272225
- Wall time: 25min 44s (local)
- 評価結果  : 1.5133521

#### 8 (157)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1093728644236824
- Wall time: 10min 4s
- 評価結果  : 1.4925832

#### 7 (235)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.108590082437519
- Wall time: 12min 56s
- 評価結果  : 1.4946937

#### 6 (207)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1135
- Best CV score: 1.1086837935965734
- Wall time: 11min 54s
- 評価結果  : 1.3999194

#### 5-tune (参考)
- 'num_leaves' : 18,
- 'learning_rate' : 0.05,
- Best num_boost_round: 2395
- Best CV score: 1.1103399384119979
- Wall time: 18min 29s (n1-standard-8)

#### 5-tune (135)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- CV score: 1.1112402566616455
- 評価結果  : 1.3991780
- CV Wall time: 27min 48s (local) -> 18min 24s (n1-standard-4) -> 9min 3s (n1-standard-8)

#### 5 (135)
- 'num_leaves' : 31,
- 'learning_rate' : 0.1,
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806